In [15]:
import requests
import pandas as pd
from datetime import datetime, timedelta
# df=pd.read_csv('/content/ind_nifty50list.csv')
df=pd.read_csv('complete.csv')
df_eq=pd.read_csv('/content/EQUITY_L (1).csv')
df_eq_500=pd.read_csv('/content/ind_nifty500list.csv')

names_to_search = [row['Symbol'] for index, row in df_eq_500.iterrows()]
names_to_search=names_to_search[:4]
names_to_search

['360ONE', '3MINDIA', 'ABB', 'ACC']

In [16]:
# -*- coding: utf-8 -*-
"""historical_data_in_simple_table_for_n_years.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1RJSU82yAjVUV06cGK7IchzBvXZ3Bk1yc
"""

import requests
import pandas as pd
from datetime import datetime

# Function to fetch intraday 1-minute data
def fetch_intraday_data(symbol):
    url = f'https://api.upstox.com/v2/historical-candle/intraday/{symbol}/1minute'
    headers = {'Accept': 'application/json'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json().get('data', [])
        return pd.DataFrame(data)
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# # Define the symbols for which you want to fetch intraday data
# symbols = ['NSE_EQ%7CINE848E01016']

# Function to fetch instrument_keys based on a list of names using boolean indexing
def get_instrument_keys(df, names):
    instrument_keys = {}

    for name in names:
        mask = df['tradingsymbol'] == name
        if mask.any():
            instrument_key = df.loc[mask, 'instrument_key'].iloc[0]
            instrument_keys[name.lower()] = instrument_key
        else:
            print(f"No instrument_key found for name: {name}")

    return instrument_keys

instrument_keys_result = get_instrument_keys(df, names_to_search)
# Dictionary to store DataFrames
dfs = {}

# Loop through each symbol and fetch intraday data
for name, instrument_key in instrument_keys_result.items():
    df_symbol = fetch_intraday_data(instrument_key)

    # Convert timestamp column to datetime format
    if 'timestamp' in df_symbol.columns:
        df_symbol['timestamp'] = pd.to_datetime(df_symbol['timestamp'])

    # Store the DataFrame in the dictionary with a name like df_reliance
    dfs[f"df_{name.lower()}"] = df_symbol

# # Access individual DataFrames using dfs dictionary
# df_nse_eq = dfs.get('df_nse_eq%7cine848e01016', pd.DataFrame())


# Function to clean the data for a given DataFrame
def clean_data(df):
    # Create an empty DataFrame with the desired column names
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'open interest']
    df_cleaned_data = pd.DataFrame(columns=columns)

    # Iterate through each row in the original DataFrame
    for index, row in df.iterrows():
        # Extract data from the list
        row_data = row[0] if isinstance(row[0], list) else row[0][0]  # Assuming each row contains a list or list within a list

        # Reshape the row_data if needed
        if isinstance(row_data, list) and len(row_data) == 1:
            row_data = row_data[0]

        # Create a temporary DataFrame
        temp_df = pd.DataFrame([row_data], columns=columns[:len(row_data)])

        # Append the temporary DataFrame to the cleaned DataFrame
        df_cleaned_data = pd.concat([df_cleaned_data, temp_df], ignore_index=True)

    # Convert timestamp column to datetime format
    df_cleaned_data['timestamp'] = pd.to_datetime(df_cleaned_data['timestamp'])

    return df_cleaned_data
for name, df_symbol in dfs.items():
    cleaned_df = clean_data(df_symbol)
    dfs[name] = cleaned_df

# Access and print individual cleaned DataFrames
for name, cleaned_df in dfs.items():
    print(f"Cleaned DataFrame for {name}:")
    print(cleaned_df)

# # Example usage for each DataFrame
# df_nse_eq_cleaned = clean_data(df_nse_eq)
# df_other_symbol_cleaned = clean_data(df_other_symbol)

# # Print cleaned DataFrames
# print("Cleaned DataFrame for NSE_EQ%7CINE848E01016:")
# print(df_nse_eq_cleaned)

# print("\nCleaned DataFrame for OTHER_SYMBOL:")
# print(df_other_symbol_cleaned)



Cleaned DataFrame for df_360one:
                    timestamp    open    high     low   close volume  \
0   2024-02-08 14:50:00+05:30  664.60  664.60  663.60  664.60    362   
1   2024-02-08 14:49:00+05:30  664.75  664.80  664.05  664.60    201   
2   2024-02-08 14:48:00+05:30  664.50  664.75  663.25  664.75    687   
3   2024-02-08 14:47:00+05:30  664.50  664.50  663.95  664.45    196   
4   2024-02-08 14:46:00+05:30  664.15  664.50  663.95  664.50    374   
..                        ...     ...     ...     ...     ...    ...   
331 2024-02-08 09:19:00+05:30  648.05  649.90  647.75  648.85   1845   
332 2024-02-08 09:18:00+05:30  648.45  649.95  647.30  647.40   1624   
333 2024-02-08 09:17:00+05:30  646.75  648.55  646.40  648.05   1686   
334 2024-02-08 09:16:00+05:30  645.30  647.40  643.60  647.40    990   
335 2024-02-08 09:15:00+05:30  643.75  648.20  643.10  645.05   3083   

    open interest  
0               0  
1               0  
2               0  
3               0  
4 

In [17]:
import pandas as pd
from datetime import datetime

# Assuming you have already defined and cleaned your DataFrames: df_reliance_cleaned, df_tatasteel_cleaned, df_infy_cleaned

# Function to save DataFrames to Excel files in a loop
def save_all_to_excel(dfs):
    for df_name, df in dfs.items():
        # Remove timezone information from datetime columns
        df_no_timezone = df.copy()
        df_no_timezone['timestamp'] = df_no_timezone['timestamp'].dt.tz_localize(None)

        file_name = f"{df_name}_historical_1minute_intraday_data.xlsx"
        df_no_timezone.to_excel(file_name, index=False)
        print(f"DataFrame '{df_name}' successfully saved to Excel file: {file_name}")

# Save all DataFrames to individual Excel files
save_all_to_excel(dfs)


DataFrame 'df_360one' successfully saved to Excel file: df_360one_historical_1minute_intraday_data.xlsx
DataFrame 'df_3mindia' successfully saved to Excel file: df_3mindia_historical_1minute_intraday_data.xlsx
DataFrame 'df_abb' successfully saved to Excel file: df_abb_historical_1minute_intraday_data.xlsx
DataFrame 'df_acc' successfully saved to Excel file: df_acc_historical_1minute_intraday_data.xlsx
